## References

* [Iris Data Set](https://archive.ics.uci.edu/ml/datasets/Iris) (UCI)

In [ ]:
import numpy as np
import pandas as pd

from problem import load_split_train_test
from problem import split_feature_target
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

In [ ]:
np.random.seed(42)

In [ ]:
data_train, data_test = load_split_train_test()
categories = data_train['Species'].cat.categories

x_train, y_train = split_feature_target(data_train)
x_test, y_test = split_feature_target(data_test)

In [ ]:
def score(model, x, y):
    return f1_score(y, model.predict(x), average='macro')

pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('model', LinearSVC()),
])

grid = [{
    'model__loss': ['hinge', 'squared_hinge'],
    'model__C': np.concatenate((np.arange(0.1, 1.1, 0.1), np.arange(10, 100, 10))),
}]

search = GridSearchCV(pipeline, grid, scoring=score)
search.fit(x_train, y_train)
print(search.best_params_)

y_pred = search.best_estimator_.predict(x_test)

In [ ]:
print('F1 score: {:.2f}'.format(f1_score(y_test, y_pred, average='macro')))

matrix = confusion_matrix(y_test, y_pred)
matrix = pd.DataFrame(matrix, index=categories, columns=categories)
matrix